# Basics on Markov chain Monte Carlo and physics applications

The aim of the Markov chain Monte Carlo (MCMC) is to produce random variables obeying any probability distributions as you like. There are three main tools for MCMC and I will introduce them one by one.

* Metropolis-Hastings algorithm
* Gibbs sampler
* Hybrid Monte Carlo

## Metropolis-(Hastings) algorithm

The Metropolis-Hastings (MH) algorithm is the most important and most popular tool in MCMC, Bayesian statistics, and physics simulations. It is universal and applicable to any kinds of distributions. The other methods usually have some restrictions, so if you do not know how to use another one like a Gibbs sampler, you should choose the MH method.

### Classical Monte Carlo simulation for Ising model
As a typical example, I will present a simple Julia implimentation of the Monte Carlo simulation of the 2D Ising model on the square lattice. The Metropolis rule is simply given by the following function.

In [1]:
function Metropolis(β::Float64,ΔE::Float64)
    return (ΔE <= 0.) || (exp(-β * ΔE) > rand())
end

Metropolis (generic function with 1 method)

This function will tell us whether we will accept a new state or not according to its energy gain $\Delta E$. It is clear that acceptance/rejection will be determined probabilistically and typically the MH alrorithm has an acceptance rate of only 20-40% (c.f. in the Ising model it is much higher). I will later show you algorithms with an almost 100% acceptance rate, such as a Gibbs sampler or a hybrid Monte Carlo.

### Storing spin configurations
The most memory-effcient way to store a spin configuration is using bitwise operations. Usually we need a very low-level code for such bitwise operations (see e.g. "Hacker's Delight" by H. S. Warren), but Julia provides a nice platform called BitArray. We regard a bit 1 as spin-up ($\sigma =+1$) and 0 as spin-down ($\sigma =-1$). For 1D, we can just store an array of spins as BitArray.

In [2]:
L = 8
spin1D = trues(L)

8-element BitArray{1}:
 true
 true
 true
 true
 true
 true
 true
 true

In order to store a 2D spin configuration, Array of BitArray is more useful than 2D BitArray (you will understand later).

In [3]:
spin2D = rand(Bool, L, L)
spin2Drow = [BitVector(spin2D[n, :]) for n in 1:L]

8-element Array{BitArray{1},1}:
 Bool[false, true, false, true, false, true, false, true]  
 Bool[false, true, false, false, true, false, true, false] 
 Bool[false, false, true, false, false, true, false, false]
 Bool[true, false, true, false, false, false, true, true]  
 Bool[false, true, false, true, true, false, false, true]  
 Bool[true, true, false, false, true, false, true, false]  
 Bool[false, false, true, true, true, true, false, false]  
 Bool[false, true, true, false, true, true, true, false]   

For the futer purpose, it is useful later to store the transposed version:

In [4]:
spin2Dcolumn = [BitVector(spin2D[:, n]) for n in 1:L]

8-element Array{BitArray{1},1}:
 Bool[false, false, false, true, false, true, false, false]
 Bool[true, true, false, false, true, true, false, true]   
 Bool[false, false, true, true, false, false, true, true]  
 Bool[true, false, false, false, true, false, true, false] 
 Bool[false, true, false, false, true, true, true, true]   
 Bool[true, false, true, false, false, false, true, true]  
 Bool[false, true, false, true, false, true, false, true]  
 Bool[true, false, false, true, true, false, false, false] 

### Updating
Assuming a periodic boundary condition, the nearest-neighbor sites can be obtained by bitwise shift (rotation) operations. It is appropriate for the small $L$ but for a larger size it has a large overhead.

In [5]:
J = -1.0 # Ferro
function energygain(x, y, row, column)
    left = xor(ror(row[y],1)[x], row[y][x])
    right = xor(rol(row[y],1)[x], row[y][x])
    top = xor(ror(column[x],1)[y], column[x][y])
    bottom = xor(rol(column[x],1)[y], column[x][y])
    return -2. * J * sum(1 - 2 * [left,right,top,bottom])
end
energygain(3, 1, spin2Drow, spin2Dcolumn)

-4.0

If we choose the flipping spin sequentially (sequential sweep), it may destroy a detailed balance microscopically. Usually it is better to choose a candidate spin using a random variable. Note that functions can be used as arguments in Juila.

In [6]:
function step(β, method, row, column)
    x = rand(1:L)
    y = rand(1:L)
    ΔE = energygain(x, y, row, column)
    flip = method(β, ΔE)
    row[y][x] = xor(row[y][x], flip)
    column[x][y] = xor(column[x][y], flip)
    return(x, y, flip)
end
step(0.1, Metropolis, spin2Drow, spin2Dcolumn)

(8, 4, false)

Julia is using "call by reference" for Array, so spin2Drow will be directely updated.

In [7]:
spin2Drow

8-element Array{BitArray{1},1}:
 Bool[false, true, false, true, false, true, false, true]  
 Bool[false, true, false, false, true, false, true, false] 
 Bool[false, false, true, false, false, true, false, false]
 Bool[true, false, true, false, false, false, true, true]  
 Bool[false, true, false, true, true, false, false, true]  
 Bool[true, true, false, false, true, false, true, false]  
 Bool[false, false, true, true, true, true, false, false]  
 Bool[false, true, true, false, true, true, true, false]   

### Simulations
Let's warm up!

In [8]:
function warmup(β)
    N = 10000000
    for i in 1:N
        step(β, Metropolis, spin2Drow, spin2Dcolumn)
    end
end
warmup(0.1)
spin2Drow

8-element Array{BitArray{1},1}:
 Bool[true, false, false, true, true, false, false, false]
 Bool[true, false, false, false, true, true, true, false] 
 Bool[false, false, true, true, true, false, false, false]
 Bool[true, false, false, true, true, true, true, true]   
 Bool[false, true, true, true, true, false, true, false]  
 Bool[false, true, true, true, true, true, true, true]    
 Bool[false, false, true, false, false, true, true, false]
 Bool[true, false, false, true, false, true, true, false] 

At low temperature, the spins will be ordered to the ferromagnetic state.

In [9]:
warmup(10.0)
spin2Drow

8-element Array{BitArray{1},1}:
 Bool[false, false, false, true, true, true, true, false]
 Bool[false, false, false, true, true, true, true, false]
 Bool[false, false, false, true, true, true, true, false]
 Bool[false, false, false, true, true, true, true, false]
 Bool[false, false, false, true, true, true, true, false]
 Bool[false, false, false, true, true, true, true, false]
 Bool[false, false, false, true, true, true, true, false]
 Bool[false, false, false, true, true, true, true, false]

## Gibbs sampler (heat-bath method)
A Gibbs sampler is the most powerful tool in MCMC if you are able to generate a "conditional probability distribution." Only in this case, we can achieve a 100% acceptance rate for the continuous variable distributions, i.e. the acceptance/rejection process is deterministic. For some reason a Gibbs sampler is called heat-bath method in physics, but you should learn it in this generalized form.

### Example

Assuming we want to sample from a distribution defined by a probablity
$$
p(x,y) \propto e^\frac{-a x^2+2b xy - cy^2}{2},
$$
we can explicitly derive a conditional distribution as a Gaussian distribution for each variable.
$$
p(x|y) \propto e^{-\frac{a}{2} \left(x - \frac{b}{a}y\right)^2}, \\
p(y|x) \propto e^{-\frac{c}{2} \left(y - \frac{b}{c}x\right)^2}.
$$

By updating $x$ and $y$ alternately from these two conditional distributions, we can produce the original joint distribution because it is the invariant distribution of these two sampling processes. Note that the detailed balance is microscopically broken, but the algorithm is balanced as a total. If you wish to resotre it, it is better to choose $x$ or $y$ randomly.

In [10]:
using Distributions
function Gibbs_sampler(a::Float64, b::Float64, c::Float64, x, y)
    x = rand(Normal(b / a * x, 1 / sqrt(a)))
    y = rand(Normal(b / c * x, 1 / sqrt(c)))
    return x, y
end
Gibbs_sampler(1.0, 0.8, 1.0, 0.0, 0.0)

(-0.6954766255391048, -0.5444156831545962)

Julia includes the library for various distributions, so the implementation should be simple. After the convergence it will give us a correct distribution.

In [11]:
function sampling(a, b, c)
    N = 1000000
    x = 0.0
    y = 0.0
    for i in 1:N
        x, y = Gibbs_sampler(a, b, c, x, y)
    end
    return x, y
end
sampling(1.0, 0.8, 1.0)

(-3.3599654480688774, -3.877891808964689)

### For Ising model

For the Ising model, the Gibbs sampler will reduce to just a so-called heat-bath method probably because the unupdated spins can be regarded as a heat bath and the transition probablities obey the partial "canonical distribution" for the updated spin. The implementation is simple and can be done just by replacing the function Metropolis to the following one:

In [12]:
function heatbath(β::Float64,ΔE::Float64)
    return (1. / (exp(+β * ΔE) + 1.)) > rand()
end
step(0.1, heatbath, spin2Drow, spin2Dcolumn)

(7, 3, false)

Note that the acceptance rate is apparently not 100% due to the finite number of the possible transitions.

In [13]:
function heatup(β)
    N = 10000000
    for i in 1:N
        step(β, heatbath, spin2Drow, spin2Dcolumn)
    end
end
heatup(0.1)
spin2Drow

8-element Array{BitArray{1},1}:
 Bool[true, true, true, false, false, false, true, true]    
 Bool[false, false, false, false, false, true, true, false] 
 Bool[true, true, true, false, false, false, true, false]   
 Bool[false, false, true, true, true, false, false, false]  
 Bool[false, true, true, false, true, true, true, true]     
 Bool[false, true, false, true, true, true, false, true]    
 Bool[true, true, false, true, true, true, true, false]     
 Bool[false, true, false, false, false, false, false, false]

In [14]:
heatup(10.0)
spin2Drow

8-element Array{BitArray{1},1}:
 Bool[true, true, true, true, true, true, true, true]        
 Bool[false, false, false, false, false, false, false, false]
 Bool[false, false, false, false, false, false, false, false]
 Bool[true, true, true, true, true, true, true, true]        
 Bool[true, true, true, true, true, true, true, true]        
 Bool[true, true, true, true, true, true, true, true]        
 Bool[true, true, true, true, true, true, true, true]        
 Bool[true, true, true, true, true, true, true, true]        

## Hybrid Monte Carlo

Condensed matter physicists do not usually use this method because they are mostly using discrete variables, but this method is the most efficient method with an almost 100% acceptance rate "for continuous variables" because it is using a gradient of the probability distribution. A hybrid Monte Carlo or Hamiltonian Monte Carlo (HMC) would be the best solution for the distribution with a known gradient and it is much more efficient than the pure MH method because of the highest acceptance rate.

The essence of HMC can be understood as some hybrid of the MH method and the Gibbs sampler (or deterministic molecular dynamics), and HMC generates an extended probability distribution including an auxiliary field called "momentum," where the conditional probability distribution without this momentum becomes the disired one. In some quantum Monte Carlo (QMC) simulations, it is difficult to calculate the gradient explicitly, and then we have to introduce another auxiliary field (called "pseudofermion" although it is actually bosonic) to estimate the gradient probabilistically "as an avarage."

### How it works

Instead of generating the original distribution $\exp(-V(\chi))$ (e.g. $V(\chi) = \beta E(\chi)$ for the continuous variables $\chi$), we would like to generate a joint distribution of $\chi$ and $\pi$ (if $\chi$ is a vector, $\pi$ should have the same dimention):
$$p(\chi,\pi)=\exp(-\pi^2-V(\chi)).$$
If we simply discard the auxiliary variables $\pi$, the resulting conditional distribution simply becomes the original one.
$$p(\chi | \pi)=\exp(-V(\chi)).$$
Thus it is enough to generate the extended distribution with doubled variables.

If we define a classical Hamiltonian as $\mathcal{H}=\pi^2+V(\chi)$, the problem results in the generation of the distribution of the classical probabilistic system with $p=\exp(-\mathcal{H})$. Note that we simply set the mass $m=1/2$, but we can generalize it to any mass matrix.

### Algorithm

The updating process goes similarly to the Gibbs sampler. First, we update $\pi$ from
$$p(\pi | \chi)=\exp(-\pi^2).$$
This is just a normal distribution, so Box–Muller's method simply works for this step.

We use a deterministic molecular dynamics to update $\chi$. Since the equation of motion conserves the total energy, the distribution is not changed by this motion from $t=0$ to any time $t=T$ when we update both $\chi$ and $\pi$ simultaneously according to the canonical equation. Typically, we move $(\chi,\pi)$ for $T=1$ according to the randomly selected momentum $\pi$ in order to move $\chi$ for a long enough distance (to avoid the self-correlation). The detailed balance completely holds for this deterministic updating *as long as the equation of motion is solved exactly without any numerical error.* Under this (numerically impossible) assumption, we can achieve 100% acceptance rate for this HMC algorithm.

### Leap-flogging

In the ideal case with a negligible numerical error, HMC can achieve almost 100% acceptance rate and in this sense it is very efficient, but this is impossible in a real computer using floating-point numbers. Thus, we use the following integrator for the equation of motion to keep its detailed balance:

1. $\chi(t+\Delta t/2)  = \chi(t) + \pi(t) \Delta t.$
1. $\pi(t+ \Delta t) = \pi(t) - \nabla V(\chi+\Delta t/2) \Delta t.$
1. $\chi(t+\Delta t)  = \chi(t+\Delta t/2) + \pi(t+ \Delta t) \Delta t.$

This algorithm is apparently reversible, i.e. everything goes back when we replace $\pi \to -\pi$. This fact will not be affected by the numerical error because the unupdated variable is conserved for each step (see e.g. the lifting algorithm for the wavelet transformation). Thus, the detailed balance (i.e. reversibility of the process) is still kept even with a numerical error for this leap-flogging approximation considering the fact that the normal distribution produces the same amount of $\pi$ and $-\pi$.

However, if $\Delta t$ is not small enough, the energy $\mathcal{H}$ is no longer conserved due to the numerical error and we need an acceptance/rejection process to keep its distribution. This is simply realized by the MH algorithm for the accumulated error for the energy $\Delta \mathcal{H} = \mathcal{H}_\textrm{new} - \mathcal{H}_\textrm{old}$. Note that we are dealing with a probability density, not a probability itself, so the measure $d\chi d\pi$ of $p(\chi,\pi)d\chi d\pi$ also has to be conserved. The leap-flogging has no problem on this point because it clearly conserves the symplectic form $d\chi d\pi$ regardless of the error (take the exact differential of each step).

If we begin from the MH algorithm (other way around), HMC can be regarded as an improved version of the random walk + MH algorithm. If we just do a random walk (with a momentum $\pi$) and check it by MH, there is no reason to think that the energy difference $\Delta \mathcal{H} = \mathcal{H}_\textrm{new} - \mathcal{H}_\textrm{old}$ becomes small. However, if we use a molecular dynamics instead of the random walk, the origin of $\Delta \mathcal{H}$ is totally the numerical error and we can expect that it becomes small if the calculation is accurate enough. HMC is one simple way to increase the acceptance rate of the MH algorithm to almost 100%.

### Implementation

In [15]:
function leapflogging(χ, π, Δt::Float64, N, ∇V)
    χ += π * Δt
    for i in 1:(N-1)
        π -= ∇V(χ) * Δt
        χ += π * 2Δt
    end
    π -= ∇V(χ) * Δt
    χ += π * Δt
    return χ, π
end

leapflogging (generic function with 1 method)

In [16]:
function HMC(χ::Vector{Float64}, V::Function, ∇V::Function)
    Nstep = 1000
    Nintegrate = 10
    Δt = 1. / Nintegrate
    π = similar(χ)
    for i in 1:Nstep
        π = rand(Normal(0, 1 / sqrt(2)), length(π))
        Hold = dot(π, π) + V(χ)
        χold = χ
        πold = π
        χ, π = leapflogging(χ, π, Δt, Nintegrate, ∇V)
        Hnew = dot(π, π) + V(χ)
        println((Hold, Hnew))
        if Metropolis(1., Hnew - Hold)
            println("Accepted!")
        else
            χ = χold
            π = πold
        end
    end
    return χ
end
v = x -> (x[1] ^ 2 - 2 * 0.8x[1] * x[2] + x[2]^2) / 2.
dv = x -> [x[1] - 0.8x[2], x[2] - 0.8x[1]]
HMC([0., 0.], v, dv)

(1.0516311749548504, 1.047738978603347)
Accepted!
(2.054551936021356, 2.0513099434248137)
Accepted!
(2.249494195034779, 2.252790540414616)
Accepted!
(1.0557769045590966, 1.0568015389457526)
Accepted!
(1.1188975748432364, 1.1177587009333996)
Accepted!
(1.158569402458434, 1.1622751687465072)
Accepted!
(0.216075725844885, 0.21586274477037373)
Accepted!
(2.617045645802097, 2.6153737854250747)
Accepted!
(0.6403287044098094, 0.6416795202449006)
Accepted!
(4.0118729659917705, 3.986790873662178)
Accepted!
(3.7063421749649255, 3.7302836150519694)
Accepted!
(0.8988005470283749, 0.8999646927021241)
Accepted!
(1.1574051625641006, 1.156043021783097)
Accepted!
(1.9364734468404292, 1.934186629794401)
Accepted!
(2.7267924425093457, 2.725179929118486)
Accepted!
(1.1695633983802107, 1.1746895364942502)
Accepted!
(2.4959858930710404, 2.4948706002825594)
Accepted!
(2.3626655763453, 2.3609362524245707)
Accepted!
(1.654039072735686, 1.656779735582939)
Accepted!
(1.796357070549493, 1.7965644557187028)
Accept

Accepted!
(0.7547870082865824, 0.7532877897516919)
Accepted!
(0.5502752196069463, 0.5525969760351076)
Accepted!
(1.0482800856903263, 1.0451440472312874)
Accepted!
(1.0111643897500362, 1.014331748594262)
Accepted!
(1.033834582819066, 1.0281933540285022)
Accepted!
(2.163994642523793, 2.1696654789296734)
Accepted!
(1.6228031202628608, 1.620767798997617)
Accepted!
(7.085037557715661, 7.086583921967023)
Accepted!
(0.7468682287095288, 0.7466400340363141)
Accepted!
(1.6624720076059603, 1.6618257205118332)
Accepted!
(0.9166254767605078, 0.9149847693376361)
Accepted!
(0.7317122168285268, 0.7320683675800624)
Accepted!
(1.066765299508381, 1.0606552785499568)
Accepted!
(1.2406621427825373, 1.2485949070516515)
Accepted!
(1.3914300875017052, 1.391218690606777)
Accepted!
(2.6910303724305074, 2.676655418591178)
Accepted!
(3.300526311095115, 3.2975360030723597)
Accepted!
(3.2617722090325425, 3.2780037489684197)
Accepted!
(0.32901248998097415, 0.32987384840374195)
Accepted!
(0.27984991561206535, 0.28084

(1.7432951730980046, 1.7474844389179014)
Accepted!
(1.2379253656684575, 1.2307596490332835)
Accepted!
(1.3639145658052967, 1.3653879783335874)
Accepted!
(3.513561162051801, 3.5149702180840423)
Accepted!
(1.0050763831586864, 1.00958374432975)
Accepted!
(2.2516193634836847, 2.249522495875473)
Accepted!
(2.2788584559918945, 2.280983753609194)
Accepted!
(0.8824913573056492, 0.882613366796534)
Accepted!
(0.07636267073619257, 0.07626654502912067)
Accepted!
(1.5427886627654355, 1.5358291652455356)
Accepted!
(1.9900979255296605, 1.9854984997602716)
Accepted!
(2.33541132867288, 2.3322032430076107)
Accepted!
(3.489587212798166, 3.4817998708297573)
Accepted!
(2.726811984198023, 2.745232120549976)
Accepted!
(1.272535282244263, 1.2711158697167275)
Accepted!
(1.5809279339990518, 1.583161596080941)
Accepted!
(1.044187302933656, 1.0466186643040791)
Accepted!
(0.5688604706534945, 0.5694251897976947)
Accepted!
(0.7511261834760538, 0.7510691748423957)
Accepted!
(1.9604882151417153, 1.952175038744839)
Acc

(3.0103790806582755, 3.031865273382799)
Accepted!
(0.5709241489423283, 0.5711896665533731)
Accepted!
(1.1420509584480705, 1.1359404648343496)
Accepted!
(2.106327326251149, 2.104703268472675)
Accepted!
(1.4949103627798852, 1.494721305421172)
Accepted!
(1.2414657244426026, 1.248590285337912)
Accepted!
(5.889661801806191, 5.888964405620579)
Accepted!
(1.4815833882195353, 1.4822538891575907)
Accepted!
(4.856495732529569, 4.853397807074796)
Accepted!
(4.573412678387177, 4.573149481916313)
Accepted!
(4.36420890960609, 4.366118222510925)
Accepted!
(2.927620395212482, 2.925426705596028)
Accepted!
(2.9577437029433558, 2.956810221780716)
Accepted!
(5.585794865208086, 5.572210384490658)
Accepted!
(5.545268008213264, 5.56381221582205)
Accepted!
(3.3762294395966137, 3.3598207105360496)
Accepted!
(3.4335466908297994, 3.448571600832529)
Accepted!
(1.5243214363690365, 1.5264996513963545)
Accepted!
(0.38478543623857764, 0.38481167814245265)
Accepted!
(0.7026927513285075, 0.7002605861041049)
Accepted!
(

Accepted!
(1.4179183847249448, 1.4181667195466632)
Accepted!
(1.5254432905660726, 1.5273499413980471)
Accepted!
(1.3956956318620835, 1.3947021854395385)
Accepted!
(1.2274760041528734, 1.2295967403787058)
Accepted!
(0.29014417578931434, 0.2902051488184583)
Accepted!
(1.668153308176233, 1.667185893327492)
Accepted!
(1.7194013092974183, 1.7159930457997992)
Accepted!
(1.8960520182730263, 1.8968099014922226)
Accepted!
(2.1373269174757157, 2.139052128881532)
Accepted!
(0.604978092738882, 0.6070380246326906)
Accepted!
(0.6819623275490999, 0.6772858323429801)
Accepted!
(1.6380277293056493, 1.6303428749939664)
Accepted!
(1.889015851578816, 1.891429615569812)
Accepted!
(1.4079882321531336, 1.4115673851378536)
Accepted!
(1.9580871701588092, 1.9597917660466821)
Accepted!
(1.7947550122471803, 1.7895794015169295)
Accepted!
(2.8987456378888514, 2.902359050403335)
Accepted!
(3.0811875711897203, 3.079876389204)
Accepted!
(3.293480816473242, 3.300439905714893)
Accepted!
(0.844487603626247, 0.84459779432

2-element Array{Float64,1}:
 -0.969077 
  0.0760265

### Automatic Differentiation

Now in the field of machine learning or Bayesian statistics, the "autograd" i.e. automatic differentiation becomes one of the most powerful tools. Physicists still like to do differentiations by themselves, but a functional language like Julia can differentiate the function directly, so we no longer need any paper/pen.

In [17]:
using ForwardDiff
HMC([0., 0.], v, x -> ForwardDiff.gradient(v, x))

(0.7163496131647713, 0.7123676889499415)
Accepted!
(0.6116432570284567, 0.6133309663528324)
Accepted!
(0.7199977387759098, 0.7208330678017513)
Accepted!
(0.7287734428170248, 0.7296858527675929)
Accepted!
(1.2739881257921333, 1.2737097446849122)
Accepted!
(1.158545714933612, 1.1582074202433459)
Accepted!
(5.576334230757476, 5.558719363577827)
Accepted!
(4.980988609652447, 4.994100026809242)
Accepted!
(2.7602623191580444, 2.7642732114407544)
Accepted!
(2.513054050900536, 2.5083594094902484)
Accepted!
(1.288860540114028, 1.294168967287533)
Accepted!
(1.6558902051174118, 1.655464140673352)
Accepted!
(1.7974421719371265, 1.7987663408898686)
Accepted!
(0.2705217473187372, 0.2698698264405284)
Accepted!
(1.7983277373555042, 1.7877795849592173)
Accepted!
(2.1023014793302015, 2.113147621377296)
Accepted!
(0.4319705089868965, 0.4303893802567713)
Accepted!
(0.5182061783317984, 0.5201167119323149)
Accepted!
(0.13191696777016343, 0.13145656873915854)
Accepted!
(1.1266334032572962, 1.1187201893244565

(2.3934395053536206, 2.3963793839195144)
Accepted!
(2.1423260460044733, 2.1410958248899776)
Accepted!
(3.200664382140741, 3.1986901313051215)
Accepted!
(1.995951476261403, 1.988741705495745)
Accepted!
(5.1935238785562605, 5.173019855324779)
Accepted!
(3.593493653134672, 3.6217002089213897)
Accepted!
(3.2904548461727603, 3.274846228354586)
Accepted!
(2.828209107242896, 2.846513509932513)
Accepted!
(1.5287265661327814, 1.5202222256325009)
Accepted!
(2.027524755325559, 2.027670294916452)
Accepted!
(1.7627689247791056, 1.771318040295377)
Accepted!
(0.6437015572534355, 0.6408568843547995)
Accepted!
(1.4748179071935645, 1.476477612958829)
Accepted!
(1.3357068538702015, 1.3344460339001238)
Accepted!
(2.4819794219983784, 2.4782374750526768)
Accepted!
(3.23603278200182, 3.2392698670095657)
Accepted!
(3.335909081658583, 3.333085952755634)
Accepted!
(3.7025501489647707, 3.7052451644883524)
Accepted!
(3.9131478917901026, 3.909931675091031)
Accepted!
(5.092362600001302, 5.093211652200599)
Accepted!

(1.0512590441172833, 1.0461380072142124)
Accepted!
(1.551080835701924, 1.5449148469284628)
Accepted!
(3.680246798958311, 3.6912005433832955)
Accepted!
(1.3943359797358115, 1.390626192453211)
Accepted!
(3.8753931652335543, 3.8747162098233536)
Accepted!
(2.805249130767738, 2.801876707802282)
Accepted!
(2.3042897925715624, 2.30995122176109)
Accepted!
(2.5031769169406135, 2.4936053400669977)
Accepted!
(2.078880457874912, 2.0906046473151245)
Accepted!
(1.1357655661266535, 1.1352969986670791)
Accepted!
(3.413318593462781, 3.409087040917872)
Accepted!
(3.36042003756432, 3.360242442561712)
Accepted!
(3.772885200886389, 3.7747684516135416)
Accepted!
(4.342364050047111, 4.340681008713294)
Accepted!
(5.569278763804733, 5.565721451764369)
Accepted!
(3.941577649131937, 3.9323160873150425)
Accepted!
(5.6268820230925, 5.6405167816754656)
Accepted!
(4.8953355555408535, 4.884780184451788)
Accepted!
(4.450527463258295, 4.454757397497533)
Accepted!
(1.5769033269290875, 1.5830027081028617)
Accepted!
(3.15

(1.1881066359506502, 1.1945074896328045)
Accepted!
(0.7531940481951945, 0.7512208639808935)
Accepted!
(3.3826265194002123, 3.3837029664679816)
Accepted!
(3.0320318227224874, 3.026872509299249)
Accepted!
(3.942843219572019, 3.9466290760941374)
Accepted!
(3.463073510440209, 3.4632537435026034)
Accepted!
(6.0563931775590305, 6.0279155769628945)
Accepted!
(4.947931684781174, 4.9780068494287955)
Accepted!
(0.5956438702868099, 0.5975953513188306)
Accepted!
(1.0849491737459487, 1.0815033558014018)
Accepted!
(0.9402603627514048, 0.9426019129547345)
Accepted!
(1.5523429677286418, 1.5500511074564836)
Accepted!
(0.4321435807836333, 0.4356834215452403)
Accepted!
(0.5627975316706727, 0.5584905421935636)
Accepted!
(0.9808343130241157, 0.9780225838553519)
Accepted!
(3.311522063500863, 3.305596887140369)
Accepted!
(3.2009050575372457, 3.1912659754645345)
Accepted!
(4.705563471493658, 4.724777347783188)
Accepted!
(1.2226939412590683, 1.2259502922553305)
Accepted!
(2.818425735027692, 2.8112789840349124)

2-element Array{Float64,1}:
 3.70001
 1.61309

1. https://github.com/JuliaDiff/ForwardDiff.jl